In [ ]:
import boto3 as b3
client = b3.client('s3')

#  Generate Pre-Signed Url

In [ ]:
def gen_ps_url(bucket_name, object_name):
    url = client.generate_presigned_url(ClientMethod='put_object', Params={'Bucket': bucket_name, 'Key': object_name}, ExpiresIn=3600, HttpMethod='PUT')
    return url

In [ ]:
gen_ps_url('sani-store', './app.py')

In [ ]:
import requests
from flask import Flask
app = Flask(__name__)

presigned_url = gen_ps_url('sani-store', 'app.py')

@app.route('/upload', methods=['POST'])
def upload_to_s3(presigned_url, file_path):
    with open(file_path, 'rb') as file:
        files = {'file': file}
        response = requests.put(presigned_url, data=file)
        return response

# Example usage
file_path = './app.py'
response = upload_to_s3(presigned_url, file_path)
print("Upload response status code:", response.status_code)
print("Upload response content:", response.content)

In [1]:
from flask import Flask, request, jsonify
import boto3
import requests
from threading import Thread

app = Flask(__name__)

def gen_ps_url(bucket_name, object_name):
    s3_client = boto3.client('s3')
    url = s3_client.generate_presigned_url(
        ClientMethod='put_object',
        Params={'Bucket': bucket_name, 'Key': object_name},
        ExpiresIn=3600,
        HttpMethod='PUT'
    )
    return url

@app.route('/generate_presigned_url', methods=['POST'])
def generate_presigned_url():
    data = request.get_json()
    bucket_name = data.get('bucket_name')
    object_name = data.get('object_name')

    if not bucket_name or not object_name:
        return jsonify({"error": "Bucket name and object name are required"}), 400

    url = gen_ps_url(bucket_name, object_name)
    return jsonify({"presigned_url": url})

@app.route('/upload_file', methods=['POST'])
def upload_file():
    presigned_url = request.form.get('presigned_url')
    file = request.files['file']

    if not presigned_url or not file:
        return jsonify({"error": "Presigned URL and file are required"}), 400

    response = requests.put(presigned_url, data=file)
    if response.status_code == 200:
        return jsonify({"message": "File uploaded successfully"}), 200
    else:
        return jsonify({"error": "Failed to upload file", "details": response.content.decode()}), response.status_code

# Run the Flask app in a separate thread
def run_flask_app():
    app.run(debug=True, use_reloader=False)

# Create and start a thread to run the Flask app
flask_thread = Thread(target=run_flask_app)
flask_thread.start()
